# GENERAL WORKFLOW

## Impact calculation process

To calculate impacts for a specific asset, scenario, year, vulnerability model, and hazard, use the following function:
`calculate_impacts(assets, hazard_model, vulnerability_model, scenario, year) -> Dict[ImpactKey, List[AssetImpactResult]]`

### First part: calculating intensity curves
(Calculate intensities and return periods for acute hazards, parameters, and definitions for chronic hazards)

1. **Request Hazard Data**:

   For each asset, generate requests for hazard data and obtain responses:

   ```python
   asset_requests, responses = _request_consolidated(hazard_model, model_asset, scenario, year)
   ```

   ```python
   HazardDataRequest(self.hazard_type, asset.longitude, asset.latitude, scenario=scenario, year=year, indicator_id=self.indicator_id)
   ```
      responses are obtained from:

   ```python
      hazard_model.get_hazard_events(requests)
   ```


2. **Process Hazard Events**:

   If hazards are acute (events) or chronic (parameters), the responses are processed differently:
   Acute Hazards: Responses include periods, intensities, units, and paths.
   Chronic Hazards: Responses include parameters, definitions, units, and paths.


3. **Retrieve Data**:

   For Acute Hazards:

   ```python
   hazard_data_provider = self.hazard_data_providers[hazard_type]
   intensities, return_periods, units, path = hazard_data_provider.get_data(longitudes, latitudes, indicator_id, scenario, year, hint, buffer)
   ```
   For Chronic Hazards:

   ```python
   hazard_data_provider = self.hazard_data_providers[hazard_type]
   parameters, definitions, units, path = hazard_data_provider.get_data(longitudes, latitudes, indicator_id, scenario, year, hint, buffer)
   ```

   ```python
   get_data(self, longitudes: List[float], latitudes: List[float], *, indicator_id: str, scenario: str, year: int, hint: Optional[HazardDataHint] = None, buffer: Optional[int] = None)
   ```

   The ``get_data`` method retrieves hazard data for given coordinates.

4. **Determine Data Path**:

   Build the path for data retrieval:

   ```python
   path = self._get_source_path(indicator_id=indicator_id, scenario=scenario, year=year, hint=hint)
   ```

   get_source_path(SourcePath) provides the source path mappings.

5. **Retrieve Curves**:

   If buffer is None, use:

   ```python
   values, indices, units = self._reader.get_curves(path, longitudes, latitudes, self._interpolation)
   ```

   If buffer is specified (The ``buffer`` variable is used to specify an area of a given size, as indicated by this variable, instead of using a single point):

   ```python
   values, indices, units = self._reader.get_max_curves(
      path,
      [
         (
            Point(longitude, latitude)
            if buffer == 0
            else Point(longitude, latitude).buffer(
               ZarrReader._get_equivalent_buffer_in_arc_degrees(latitude, buffer)
            )
         )
         for longitude, latitude in zip(longitudes, latitudes)
      ],
      self._interpolation
   )
   ```

6. **Data Retrieval Functions**:

   ```python
   get_curves(self, set_id, longitudes, latitudes, interpolation="floor")
   ```

   Get Curves: Retrieves intensity curves for each coordinate pair. Returns intensity curves, return periods, and units.

   First, it constructs the path used to select the corresponding data in the bucket. From this data, it extracts the transformation matrix, coordinate system, data units, and return periods or indices (``index_values``). Next, it converts the geographic coordinates to image coordinates. Then, it interpolates the data based on the specified interpolation method.

   If the interpolation method is ``"floor"``, it converts ``image_coords`` to integer values using the floor function and adjusts coordinates for wrapping around the dataset dimensions. It retrieves the data values using ``z.get_coordinate_selection``, then reshapes and returns the data along with ``index_values`` and ``units``.

   For other interpolation methods (``"linear"``, ``"max"``, ``"min"``), it calls ``_linear_interp_frac_coordinates`` to perform the specified interpolation. Finally, it returns the interpolated results along with ``index_values`` and ``units``.

   ```python
   get_max_curves(self, set_id, shapes, interpolation="floor")
   ```

   Get Max Curves: Retrieves the maximum intensity curves for given geometries. Returns maximal intensity curves, return periods, and units.

   First, it constructs the path used to locate the corresponding data in the bucket, similar to the ``get_curves`` method. From this data, it extracts the transformation matrix, coordinate system, data units, and index values (``index_values``). It then computes the inverse of the affine transformation matrix and applies it to the input geometries, transforming them into the coordinate system of the dataset.

   Next, it generates a ``MultiPoint`` for each shape by creating a grid of points within the shape's bounding box and intersecting these points with the shape to retain only those points that lie within the shape. If the intersection of a shape with the grid points is empty, it falls back to using the centroid of the shape as a single point.

   For the ``"floor"`` interpolation method, it converts the transformed coordinates to integer values using the floor method, retrieves the corresponding data values, and reshapes the data. For other interpolation methods (``"linear"``, ``"max"``, ``"min"``), it combines the transformed shapes with the multipoints and computes the fractional coordinates for interpolation.

   Finally, it calculates the maximum intensity values for each shape by grouping the points corresponding to each shape and finding the maximum value for each return period. The method then returns the maximum intensity curves, return periods, and units.

### Second part: applying a vulnerability model to obtain impacts

When applying a chronic-type vulnerability model, the impact is calculated using the model's `get_impact` method. This method will return an `ImpactDistrib` object, which includes `impact_bins`, `impact_type`, `path`, and `prob` (i.e., it provides the impact distribution along with the hazard data used to infer it). This result is then stored in an `AssetImpactResult` object, together with the hazard_data (which consists of the intensity curves obtained previously). The `AssetImpactResult` is subsequently saved in the results dictionary, associated with an `ImpactKey` that comprises the `asset`, `hazard_type`, `scenario`, and `year`.

On the other hand, for acute-type vulnerability models, the impact is calculated using the `get_impact_details` method of the model. This method returns an `ImpactDistrib` object, a `VulnerabilityDistrib` object (which includes `impact_bins`, `intensity_bins`, and `prob_matrix`), and a `HazardEventDistrib` object (which contains `intensity_bin_edges` and `prob`). In other words, it provides the impact distribution along with the vulnerability and hazard event distributions used to infer it. This information is stored in an `AssetImpactResult` object, which is then added to the results dictionary with an `ImpactKey`.

## Risk measures calculation process

To calculate risk measures for a specific asset, scenario, year, vulnerability model, and hazard, use the following function:
`def calculate_risk_measures(self, assets: Sequence[Asset], prosp_scens: Sequence[str], years: Sequence[int]):`

1. **Calculate all impacts**

   First, using the `_calculate_all_impacts` method, the impacts for the specific hazard, asset, and vulnerability model are calculated for all the years and scenarios. This method uses `_calculate_single_impact`, which calculates each impact using the `calculate_impacts` method previously described.

2. **Calculate risk measure**

   For each asset, scenario, year, and hazard, the corresponding impact is used to determine the risk measures according to the selected calculation method.

   The impact of the historical scenario is chosen as the `base impact`, and `risk measures` are calculated using the `calc_measure` function.

   In the default use case, the `calc_measure` method defined in the `RealEstateToyRiskMeasures` class performs calculations differently depending on whether the hazard is chronic heat or another type. The difference between the two methods is that `calc_measure_cooling` uses `mean impacts` for calculations, while `calc_measure_acute` uses `exceedance curves`. In both cases, a `Measure` object is returned, which contains a `score` (REDFLAG, HIGH, MEDIUM, LOW), `measures_0` (future_loss), and a `definition`.

   - **For cooling hazards**: It calculates the change in mean impact between historical and future scenarios. It assigns a risk score based on the future cooling levels and the change compared to predefined thresholds, returning a `Measure` object with the assigned score and future cooling value.
   
   - **For acute hazards**: It calculates the potential loss based on a 100-year return period by comparing historical and future loss values derived from exceedance curves. It assigns a risk score based on future loss levels and the change in loss relative to predefined thresholds, returning a `Measure` object with the assigned score and future loss value.

   - **For the stress_test use case**: The `calc_measure` function in the `ThermalPowerPlantsRiskMeasures` class creates a `StressTestImpact` object to obtain the percentiles (norisk, p50, p75, p90), which are used to evaluate the impact based on its `mean_intensity`. This method also returns a `Measure` object with a `score` (HIGH, MEDIUM, LOW, NORISK, NODATA), `measures_0` (mean_intensity), and a `definition`.

   - **For the generic use case**: In the `GenericScoreBasedRiskMeasures` class, the `calc_measure` method calculates risk scores differently based on whether the impact distribution is necessary or if underlying hazard data can be used instead. To generate the scores, bounds are defined for each hazard type.

     - **When using hazard data**: It compares hazard parameters to the predefined threshold bounds. It returns a score based on the severity of the hazard, or NODATA if the parameter is invalid.
   
     - **Otherwise**: The method calculates two impact measures from historical and future data. It then determines the score category based on whether these measures fall within predefined ranges and returns a `Measure` object with the score and the first measure value.
